### Projeto Final de Spark
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

#### Enviar os dados para o hdfs

No terminal:

agomes:~/spark/input$ sudo mkdir desafio

agomes:~/spark/input/desafio$ sudo curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agome:~/spark/input/desafio$ sudo unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agomes:~/spark/input/desafio$ docker exec -it namenode bash

    root@namenode:/# hdfs dfs -mkdir -p /user/arthur/desafio
    root@namenode:/# hdfs dfs -put /input/desafio/* /user/arthur/desafio
    root@namenode:/# hdfs dfs -ls /user/arthur/desafio

In [1]:
!hdfs dfs -ls /user/arthur/desafio

Found 6 items
-rw-r--r--   3 root supergroup   62492959 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv
drwxr-xr-x   - root supergroup          0 2021-07-09 00:01 /user/arthur/desafio/covid_spark
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive


In [2]:
!hdfs dfs -rm /user/arthur/desafio/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

rm: `/user/arthur/desafio/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar': No such file or directory


#### Otimizar todos os dados do hdfs para uma tabela Hive particionada por município

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [12]:
covid = spark.read.csv("/user/arthur/desafio/*.csv", header='true',sep=';')

In [13]:
covid = covid.na.fill({'coduf': 0})

In [14]:
#covid.select(isnan('municipio')).filter("isnan(municipio) = true").show()

In [15]:
covid = covid.withColumnRenamed("interior/metropolitana", "interiorOUmetropolitana")
covid = covid.withColumn("col_partition", col("coduf"))

In [16]:
covid.describe()

DataFrame[summary: string, regiao: string, estado: string, municipio: string, coduf: string, codmun: string, codRegiaoSaude: string, nomeRegiaoSaude: string, data: string, semanaEpi: string, populacaoTCU2019: string, casosAcumulado: string, casosNovos: string, obitosAcumulado: string, obitosNovos: string, Recuperadosnovos: string, emAcompanhamentoNovos: string, interiorOUmetropolitana: string, col_partition: string]

In [17]:
#!hdfs dfs -rm -r /user/arthur/desafio/covid_spark
covid.write.csv('/user/arthur/desafio/covid_spark', header='true')

In [18]:
!hdfs dfs -ls /user/arthur/desafio/covid_spark

Found 13 items
-rw-r--r--   2 root supergroup          0 2021-07-09 06:27 /user/arthur/desafio/covid_spark/_SUCCESS
-rw-r--r--   2 root supergroup   22280175 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00000-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv
-rw-r--r--   2 root supergroup   22282685 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00001-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv
-rw-r--r--   2 root supergroup   22286237 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00002-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv
-rw-r--r--   2 root supergroup   22243026 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00003-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv
-rw-r--r--   2 root supergroup   22257577 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00004-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv
-rw-r--r--   2 root supergroup   22250891 2021-07-09 06:27 /user/arthur/desafio/covid_spark/part-00005-1199e39d-03fb-4d68-a554-9b0eacb6b528

In [20]:
!hdfs dfs -tail /user/arthur/desafio/covid_spark/part-00010-1199e39d-03fb-4d68-a554-9b0eacb6b528-c000.csv

45872,629,4169,13,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-23,52,3015268,246397,525,4181,12,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-24,52,3015268,246949,552,4188,7,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-25,52,3015268,247278,329,4192,4,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-26,52,3015268,247928,650,4198,6,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-27,53,3015268,248543,615,4201,3,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-28,53,3015268,249736,1193,4223,22,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-29,53,3015268,250457,721,4241,18,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-30,53,3015268,251135,678,4248,7,"","",1,53
Centro-Oeste,DF,Brasília,53,530010,53001,DISTRITO FEDERAL,2020-12-31,53,3015268,251701,566,4259

In [21]:
#!hdfs dfs -rm -r /user/arthur/desafio/hive

Deleted /user/arthur/desafio/hive


No terminal

agomes:~/spark$ docker exec -it hive-server bash

    root@hive_server:/opt# beeline -u jdbc:hive2://localhost:10000
    
    0: jdbc:hive2://localhost:10000> create database arthur;
    0: jdbc:hive2://localhost:10000> show databases;
    +----------------+
    | database_name  |
    +----------------+
    | arthur         |
    | default        |
    +----------------+

    0: jdbc:hive2://localhost:10000> use arthur;
       
    0: jdbc:hive2://localhost:10000> create external table covid(regiao string,estado string,municipio string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string, col_partition int) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive/sem_partition' tblproperties("skip.header.line.count"="1");
    
    #Para ver estrutura
    0: jdbc:hive2://localhost:10000> desc formatted covid; 
    
    0: jdbc:hive2://localhost:10000> load data inpath '/user/arthur/desafio/covid_spark/*.csv' overwrite into table covid;
    
    0: jdbc:hive2://localhost:10000> select * from covid limit 10;
    
    #Para permitir Particionamento Dinâmico
    0: jdbc:hive2://localhost:10000> SET hive.exec.dynamic.partition = true;

    0: jdbc:hive2://localhost:10000> SET hive.exec.dynamic.partition.mode = nonstrict;
    
    0: jdbc:hive2://localhost:10000> create external table covid_otm(regiao string,estado string, municipio string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string) partitioned by (col_partition int) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive/otm' tblproperties("skip.header.line.count"="1");
    
    0: jdbc:hive2://localhost:10000> insert overwrite table covid_otm partition (col_partition) select * from covid;


In [24]:
!hdfs dfs -ls /user/arthur/desafio/hive/otm

Found 30 items
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=0
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=1
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=11
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=12
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=13
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=14
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=15
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=16
drwxr-xr-x   - root supergroup          0 2021-07-09 06:34 /user/arthur/desafio/hive/otm/col_partition=17
drwxr-xr-x   - root supergroup   